In [ ]:
# %pip install --upgrade openai

# ** Step 1 ** 

### Import Library

In [35]:
import os
from openai import OpenAI
from dotenv import load_dotenv

In [47]:
load_dotenv()

python-dotenv could not parse statement starting at line 4
python-dotenv could not parse statement starting at line 23
python-dotenv could not parse statement starting at line 24
python-dotenv could not parse statement starting at line 26
python-dotenv could not parse statement starting at line 27
python-dotenv could not parse statement starting at line 28
python-dotenv could not parse statement starting at line 29
python-dotenv could not parse statement starting at line 31
python-dotenv could not parse statement starting at line 32
python-dotenv could not parse statement starting at line 33
python-dotenv could not parse statement starting at line 34
python-dotenv could not parse statement starting at line 35
python-dotenv could not parse statement starting at line 36
python-dotenv could not parse statement starting at line 37
python-dotenv could not parse statement starting at line 39
python-dotenv could not parse statement starting at line 40
python-dotenv could not parse statement s

True

In [48]:
addendum_page_json_promt = os.getenv("ADDENDUM_PAGE_JSON")
print(addendum_page_json_promt)

None


### 환경변수 설정

In [37]:
""" 환경변수 설정 """
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
assistant_id = os.getenv('ASSISTANT_ID')
vector_store_id = os.getenv('VECTOR_STORE_ID')
base_prompt = os.getenv('BASE_PROMPT')

In [23]:
def print_status(message, status="INFO"):
    """ 터미널 메세지 커스텀(선택)"""
    status_color = {
        "INFO": "\033[94m", # blue
        "SUCCESS": "\033[92m", # green
        "WARING": "\033[93m", # yellow
        "ERROR": "\003[91m", # red
        "RESET": "\033[0m" # reset
    }

    color = status_color.get(status, status_color["INFO"])
    reset = status_color["RESET"]

    print(f"{color}[{status}] {message}{reset}")

### 파일 업로드 후 벡터 스토어에 추가 O

In [24]:
def upload_file_to_vector_store(file_paths):
    # 파일 업로드
    uploaded_file_ids = []

    for file_path in file_paths:
        if os.path.exists(file_path):
            try :
                with open(file_path, 'rb') as f :
                    uploaded_file = client.files.create(
                        file=f,
                        purpose='assistants'
                    )

                print_status(f"파일 업로드 완료: {uploaded_file.id}", "SUCCESS")


                # 벡터 스토어에 추가
                client.vector_stores.files.create(
                    vector_store_id = vector_store_id,
                    file_id = uploaded_file.id
                )

                print_status(f"벡터 스토어에 추가 완료: {uploaded_file.id}", "SUCCESS")
                uploaded_file_ids.append(uploaded_file.id)


            except Exception as e :
                print_status(f"파일을 찾을 수 없습니다: {file_path}", "ERROR")
                continue
    #업로드된 파일 개수 확인
    print_status(f"총 {len(uploaded_file_ids)}개의 파일이 벡터 스토어에 업로드 되었습니다", "SUCCESS")
    return uploaded_file_ids


# 로컬에 있는 3장 pdf 파일과 목차 경로 정의
from pathlib import Path

chat3_index_docs_path = Path(r"C:\Users\user\Desktop\Project\Ontology\Ontology_Root\목차.txt")
actions_level_docs_path = Path(r"C:\Users\user\Desktop\Project\Ontology\\Ontology_Root\위기경보수준별조치사항.pdf")
docs_path = [chat3_index_docs_path, actions_level_docs_path]

uploaded_file_ids = upload_file_to_vector_store(docs_path)

[SUCCESS] 파일 업로드 완료: file-A7RaBghofZ7vGvkUgkDkbb
[SUCCESS] 벡터 스토어에 추가 완료: file-A7RaBghofZ7vGvkUgkDkbb
[SUCCESS] 파일 업로드 완료: file-9A1F1xMtoSbMob12yYDoGU
[SUCCESS] 벡터 스토어에 추가 완료: file-9A1F1xMtoSbMob12yYDoGU
[SUCCESS] 총 2개의 파일이 벡터 스토어에 업로드 되었습니다


### 스레드 생성 및 메세지 추가 완료 O

In [25]:
def create_thread(client=client):
    thread = client.beta.threads.create()
    print_status(f"스레드 생성 완료: {thread.id}", "SUCCESS")

    thread_id = thread.id
    return thread.id

thread_id = create_thread()

C:\Users\user\AppData\Local\Temp\ipykernel_55848\103296605.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()


[SUCCESS] 스레드 생성 완료: thread_rU6ONx5oWZy8k6HsHdIeMRqh


In [26]:
def add_message_to_thread(client, thread_id, base_prompt, uploaded_file_ids):
    message_content = base_prompt + "\n\n 첨부된 파일들을 분석하여 TTL 파일을 생성해주세요. 생성 후에는 첨부된 PDF 문서를 한 페이지씩 읽어서, subclass를 보완하여 추가해주세요. 이 때 기존의 요소들은 변경하지 않고 유지한 상태로 추가만 진행합니다."

    thread_message = client.beta.threads.messages.create(
        thread_id=thread_id,\
        role = "user",
        content=message_content,
        attachments=[
            {
                "file_id": file_id,
                "tools": [{"type": "file_search"}]
            }

            for file_id in uploaded_file_ids
        ]
    )

    print_status(f"스레드 메세지 추가 완료", "SUCCESS")
    return thread_message.id

add_message_to_thread(client, thread_id, base_prompt, uploaded_file_ids)

C:\Users\user\AppData\Local\Temp\ipykernel_55848\3398931413.py:4: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread_message = client.beta.threads.messages.create(


[SUCCESS] 스레드 메세지 추가 완료


'msg_PqUMRn1ocJ6NAuQXP7Fk6RHQ'

### Assistant 실행 O

run_status.status == 'queued'        # 대기 중  
run_status.status == 'in_progress'   # 실행 중  
run_status.status == 'completed'     # 완료  
run_status.status == 'failed'        # 실패  
run_status.status == 'cancelled'     # 취소됨  
run_status.status == 'expired'       # 만료됨  
run_status.status == 'requires_action'  # 추가 액션 필요

Assistatnt를 실행하면 추가 vector stores가 생성됨

In [27]:
import time

def run_assistant_create_class(client, thread_id, base_prompt):
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
        instructions="""
                        목차를 기준으로 다른 문서들을 분석하여 OWL/RDF 표준에 맞는 TTL 파일을 생성해주세요. 

                        중요: 일관된 결과를 위해 다음을 반드시 준수하세요:
                        1. 클래스 생성 순서를 절대 변경하지 마세요
                        2. 네이밍 규칙을 절대 변경하지 마세요  
                        3. 계층 구조를 절대 변경하지 마세요
                        4. 동일한 패턴으로 모든 클래스를 생성하세요

                        TTL 파일은 Turtle 형식으로 작성하고, 적절한 네임스페이스와 클래스, 속성을 정의해주세요.
                        결과물은 한 번에 출력해주세요.
                    """ + base_prompt
    )

    while True:
        run_status = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run.id
        )

        if run_status.status == 'completed':
            print_status("Assistant 실행 완료", "SUCCESS")
            break
        
        elif run_status.status == 'failed':
            print_status("Assistant 실행 실패", "ERROR")

        elif run_status.status == 'requires_action':
            print_status("추가 액션 필요", "WARNING")
        
        elif run_status.status == 'in_progress':
            print_status("Assistant 실행 중...", "INFO")

            time.sleep(10)

    message = client.beta.threads.messages.list(
        thread_id=thread_id,
        order="desc"
    )

    return message

ontology_message = run_assistant_create_class(client, thread_id, base_prompt)
print([str(ontology_message)][:10])


C:\Users\user\AppData\Local\Temp\ipykernel_55848\1574251843.py:4: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(
C:\Users\user\AppData\Local\Temp\ipykernel_55848\1574251843.py:22: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_status = client.beta.threads.runs.retrieve(


[INFO] Assistant 실행 중...
[INFO] Assistant 실행 중...
[INFO] Assistant 실행 중...
[SUCCESS] Assistant 실행 완료


C:\Users\user\AppData\Local\Temp\ipykernel_55848\1574251843.py:42: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.list(


['SyncCursorPage[Message](data=[Message(id=\'msg_SaJLLduHB93VZmPNBJpoAepC\', assistant_id=\'asst_W7kaWMqWblyhhVj1rCP0mVow\', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value=\'아래는 첨부된 목차와 PDF 문서의 내용을 토대로 작성한 OWL/RDF Turtle (.ttl) 파일의 초안입니다. \\n\\n- 클래스 생성 순서, 네이밍 규칙, 계층구조는 목차의 순서 및 명칭을 따랐으며,\\n- 동일 패턴으로 모든 클래스를 생성하였고,\\n- PDF 문서에서 확인된 상세 조치, 역할, 상황 등의 subclass, 속성을 추후 페이지별 추가 보완 가능합니다.\\n\\n---\\n\\n```turtle\\n@prefix : <http://example.org/ontology#> .\\n@prefix owl: <http://www.w3.org/2002/07/owl#> .\\n@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\\n\\n:위기경보수준별조치사항 a owl:Ontology .\\n\\n# 최상위 클래스 - 위기경보수준별조치사항\\n:위기경보수준별조치사항 a owl:Class ;\\n    rdfs:label "위기경보수준별조치사항"@ko ;\\n    rdfs:comment "위기경보 수준에 따른 조치사항 클래스 최상위"@ko .\\n\\n# 하위 클래스 - 1. 관심, 2. 주의, 3. 경계, 4. 심각\\n:관심 a owl:Class ;\\n    rdfs:label "관심"@ko ;\\n  

-----------

### 응답 결과 TTL 추출 O

In [16]:
import re

def _validate_basic_ttl_syntax(ttl_content):
    """
    기본적인 TTL 문법 검증
    
    Args:
        ttl_content: TTL 내용 문자열
    
    Returns:
        bool: 기본 문법이 올바른지 여부
    """
    try:
        # 기본적인 TTL 요소들 확인
        has_prefix = '@prefix' in ttl_content or '@base' in ttl_content
        has_class = 'rdfs:Class' in ttl_content or 'owl:Class' in ttl_content
        has_proper_ending = ttl_content.strip().endswith('.')
        
        # 괄호 균형 확인
        open_brackets = ttl_content.count('[')
        close_brackets = ttl_content.count(']')
        bracket_balanced = open_brackets == close_brackets
        
        # 세미콜론과 마침표 확인
        has_semicolons = ';' in ttl_content
        has_periods = '.' in ttl_content
        
        validation_result = has_prefix and has_class and has_proper_ending and bracket_balanced
        
        if not validation_result:
            print_status("기본 TTL 문법 검증 실패", "WARNING")
            print_status(f"  - Prefix 선언: {has_prefix}", "INFO")
            print_status(f"  - Class 선언: {has_class}", "INFO")
            print_status(f"  - 올바른 종료: {has_proper_ending}", "INFO")
            print_status(f"  - 괄호 균형: {bracket_balanced}", "INFO")
        
        return validation_result
        
    except Exception as e:
        print_status(f"TTL 문법 검증 중 오류: {str(e)}", "ERROR")
        return False



def extract_ttl_from_response(message):
    try:
        if not getattr(message, "data", None):
            print_status("응답 메세지 없음", "ERROR")
            return None, False

        latest = message.data[0]
        if not latest.content:
            print_status("메세지 내용 없음", "ERROR")
            return None, False

        message_text = ""
        for item in latest.content:
            if hasattr(item, "text") and item.text and hasattr(item.text, "value"):
                message_text += item.text.value

        if not message_text.strip():
            print_status("텍스트 내용을 찾을 수 없습니다.", "ERROR")
            return None, False

        ttl_patterns = [
            r"```ttl\n(.*?)\n```",         # 1
            r"```turtle\n(.*?)\n```",      # 2
            r"```\n(.*?)\n```",            # 3
            r"```ttl\s*\n(.*?)\n```",      # 4
            r"```turtle\s*\n(.*?)\n```",   # 5
        ]

        PATTERN_EXPLANATIONS = {
            1: "```ttl 코드블록에서 추출됨",
            2: "```turtle 코드블록에서 추출됨",
            3: "라벨 없는 일반 ``` 코드블록에서 추출됨",
            4: "공백 포함 ```ttl 코드블록에서 추출됨",
            5: "공백 포함 ```turtle 코드블록에서 추출됨",
            0: "코드블록 미탐지 → 전체 응답을 사용하여 추출됨(폴백)",
        }

        ttl_content = None
        matched_idx = 0

        for i, pattern in enumerate(ttl_patterns, start=1):
            match = re.search(pattern, message_text, re.DOTALL | re.IGNORECASE)
            
            if match:
                ttl_content = match.group(1).strip()
                matched_idx = i
                break
                
        if not ttl_content:
            matched_idx = 0
            print_status("TTL 블록을 찾을 수 없습니다.", "ERROR")
        
        if len(ttl_content.strip()) < 50 :
            print_status("추출된 내용이 너무 짧습니다.", "ERROR")

        if not _validate_basic_ttl_syntax(ttl_content):
            print_status("TTL 문법을 확인해보세요.", "WARNING")


        print_status(f"TTL 추출 완료 (패턴 {matched_idx}) - {PATTERN_EXPLANATIONS[matched_idx]}", "SUCCESS" if matched_idx else "WARNING")
        return ttl_content, True

    except Exception as e:
        print_status(f"TTL 추출 중 오류가 발생했습니다.: {e}", "ERROR")


ontology_ttl_content = extract_ttl_from_response(ontology_message)

# 위에서 튜플 형태로 출력되기 떄문에 str로 변환해야 정상적으로 ttl 파일이 저장됨
# ('@prefix ex:     <http://example.org/disaster#> .\n@prefix rdfs: 
# ->  @prefix ex:     <http://example.org/disaster#> .
ontology_ttl_content_str = ontology_ttl_content[0]


[SUCCESS] TTL 추출 완료 (패턴 2) - ```turtle 코드블록에서 추출됨


### TTL 파일 생성 후 저장 O

In [17]:
def extract_filename(official_docs_path):
    """
    공공기관 문서명 추출
    """
    official_docs_name = os.path.basename(official_docs_path)
    official_docs_name_without_ext = os.path.splitext(official_docs_name)[0]
    return official_docs_name_without_ext

action_level_docs_name = extract_filename(actions_level_docs_path)

print(action_level_docs_name)

위기경보수준별조치사항


In [18]:
from datetime import datetime


def save_ttl_file(ttl_content, filename=None, output_dir="Ontology_Output"):
    try:
        
        # 저장할 디렉토리가 없을 경우 생성
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
            print_status(f"결과물을 저장할 폴더를 생성했습니다. : {output_dir}", "SUCCESS")

        # 파일명 설정
        if filename:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M")
            filename = f"{filename}_Class_{timestamp}.ttl"

        # 파일 확장자 확인
        if not filename.endswith('.ttl'):
            filename += '.ttl'

        # 전체 파일 경로
        file_path = os.path.join(output_dir, filename)

        # 내용 검증
        if not ttl_content or len(ttl_content.strip()) < 50:
            return None, False, "저장할 TTL 내용이 유효하지 않습니다."

        with open(file_path, 'w', encoding='utf-8') as f:
            # 내용 저장
            f.write(ttl_content)

            # footer
            f.write("# End of TTL File\n")

        file_size = os.path.getsize(file_path)
        file_size_kb = file_size / 1024

        print_status("TTL 파일 생성 완료", "SUCCESS")
        print_status(f"{filename} 파일 경로: {file_path}", "INFO")
        print_status(f"{filename} 파일 크기: {file_size_kb:.2f}KB", "INFO")
        print_status(f"{filename} 내용 길이: {len(ttl_content)}자", "INFO")


    except Exception as e:
        print_status(f"TTL 파일 생성 실패: {e}", "ERROR")
        return None


final_output = save_ttl_file(ontology_ttl_content_str, action_level_docs_name)

[SUCCESS] TTL 파일 생성 완료
[INFO] 위기경보수준별조치사항_Class_20250905_1621.ttl 파일 경로: Ontology_Output\위기경보수준별조치사항_Class_20250905_1621.ttl
[INFO] 위기경보수준별조치사항_Class_20250905_1621.ttl 파일 크기: 1.73KB
[INFO] 위기경보수준별조치사항_Class_20250905_1621.ttl 내용 길이: 1367자


--------

--------

# ** STEP 2 : 페이지 분리 후 Subclass 보완 추가 ** 

In [ ]:
def run_assistant_page_json(client, thread_id, base_prompt, addendum_page_json_promt=None):

    addendum_page_json_promt = os.getenv("ADDENDUM_PAGE_JSON")

    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
        instructions= base_prompt + addendum_page_json_promt
    )

    while True:
        run_status = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run.id
        )

        if run_status.status == 'completed':
            print_status("Assistant 실행 완료", "SUCCESS")
            break
        
        elif run_status.status == 'failed':
            print_status("Assistant 실행 실패", "ERROR")

        elif run_status.status == 'requires_action':
            print_status("추가 액션 필요", "WARNING")
        
        elif run_status.status == 'in_progress':
            print_status("Assistant 실행 중...", "INFO")

            time.sleep(10)

    page_json_message = client.beta.threads.messages.list(
        thread_id=thread_id,
        order="desc"
    )

    return page_json_message

In [29]:
page_json = run_assistant_page_json(client, thread_id, base_prompt)
print(page_json)

TypeError: can only concatenate str (not "NoneType") to str